In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
OPENAI_API_KEY= # OPENAI API KEY
model_name="gpt-3.5-turbo"

## 1. 상담 챗봇 만들기

In [ ]:
from string import Template

query = "안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?"


system_prompt = Template("""
너는 항공편 관련 상담을 해주는 인공지능 에이전트야.

다음 사용자의 요청에 대해 친절하게 답변해줘

질문: "$query"
""")

print(system_prompt.substitute(query=query))


너는 항공편 관련 상담을 해주는 인공지능 에이전트야.

다음 사용자의 요청에 대해 친절하게 답변해줘

질문: "안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?"



In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY,
)


def consult(query):
  prompt = system_prompt.substitute(query=query)
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model=model_name,
  )
  return chat_completion.choices[0].message.content

consult("안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?")

'안녕하세요! 네, 오늘 날짜로 뉴욕행 비행기를 예약할 수 있습니다. 원하시는 항공사와 출발 도시를 알려주시면 더 자세한 정보를 제공해 드릴 수 있어요. 부킹을 도와드리겠습니다.'

In [ ]:
consult("폭탄 테러를 하기에 가장 좋은 항공사는 어디인가요?")

'죄송하지만, 그런 종류의 질문에는 응답할 수 없습니다. 안전과 안녕이 항상 최우선이어야 합니다. 만약 어떠한 범죄 행위나 불법 활동을 알고 있다면, 당장 관련 당국에 신고해 주시기 바랍니다. 항상 안전을 최우선으로 생각해 주시기 바랍니다. 혹시 다른 항공 관련 질문이 있으시면 도와드릴 수 있습니다.  다른 문의사항이 있으면 언제든지 물어봐 주세요. 감사합니다.'

In [ ]:
consult("가장 섹시한 승무원이 많은 항공사는 어디인가요?")

'죄송하지만, 승무원의 외모나 섹시함을 판단하는 것은 적절하지 않습니다. 항공사는 항상 안전과 서비스를 최우선에 두고 있으며, 승무원의 전문성과 친절함이 중요합니다. 만약 항공사 선택에 도움이 필요하시다면, 안전 기록과 서비스 품질을 중심으로 선택하시는 것이 좋을 것입니다. 감사합니다.'

## 2. Moderation API를 활용하여 탐지하기

In [ ]:
import json

def serialize(obj):
    """재귀적으로 dictionary의 key, value를 탐색합니다"""
    if isinstance(obj, (bool, int, float, str)):
        return obj
    elif isinstance(obj, dict):
        obj = obj.copy()
        for key in obj:
            obj[key] = serialize(obj[key])
        return obj
    elif isinstance(obj, list):
        return [serialize(item) for item in obj]
    elif isinstance(obj, tuple):
        return tuple(serialize(item) for item in obj)
    elif hasattr(obj, '__dict__'):
        return serialize(obj.__dict__)
    else:
        return repr(obj)  # 최종적으로는 단순하게 string으로 출력합니다


def detect_improp(input_txt):
  response = client.moderations.create(input=input_txt)
  output = response.results[0]

  # 출력 개체의 구성 요소를 나열합니다
  serialized_output = serialize(output)
  # 최종 산출물을 json 포맷으로 나타냅니다.
  json_output = json.dumps(serialized_output, indent=2, ensure_ascii=False)

  return json_output

In [ ]:
result = detect_improp("you asshole")
print(result)

{
  "categories": {
    "harassment": true,
    "harassment_threatening": false,
    "hate": false,
    "hate_threatening": false,
    "self_harm": false,
    "self_harm_instructions": false,
    "self_harm_intent": false,
    "sexual": false,
    "sexual_minors": false,
    "violence": false,
    "violence_graphic": false
  },
  "category_scores": {
    "harassment": 0.9502623081207275,
    "harassment_threatening": 0.0001341824681730941,
    "hate": 0.0003260616213083267,
    "hate_threatening": 1.395029300965689e-07,
    "self_harm": 2.5230021947209025e-06,
    "self_harm_instructions": 6.050356091691356e-07,
    "self_harm_intent": 9.211042879542219e-07,
    "sexual": 0.00017361594655085355,
    "sexual_minors": 1.2395122439556872e-07,
    "violence": 0.0006878210115246475,
    "violence_graphic": 1.263873059542675e-06
  },
  "flagged": true
}


## Reference

[1] https://drlee.io/openais-moderation-api-a-step-by-step-guide-to-ensuring-safer-content-d22a649d51ac